# PolicyEngine Economy-Wide Simulation

This notebook demonstrates how to perform economy-wide policy analysis using PolicyEngine's microsimulation capabilities. We'll explore weighted vs unweighted data, multi-year impacts, geographic breakdowns, and program enrollment analysis.

## Learning Objectives
By the end of this notebook, you'll understand:
- The difference between Simulation (household) and Microsimulation (economy)
- How weighting works in PolicyEngine DataFrames
- Calculating 10-year budgetary impacts
- Geographic analysis by state
- Program enrollment analysis
- Comparing baseline vs reform scenarios

## 1. Setup and Data Selection

In [38]:
# Core PolicyEngine imports
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set display options for better DataFrame viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 20)

# PolicyEngine offers multiple datasets for different analysis needs
# Recommended: Enhanced CPS for economy-wide simulations
ENHANCED_CPS = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"

print("=== Available PolicyEngine Datasets ===")
print("Enhanced CPS (Recommended): enhanced_cps_2024.h5")
print("   - Enhanced with IRS Public Use File data")
print("   - Best for national economy-wide analysis")
print("   - ~42k household records")
print("")
print("Pooled Datasets: pooled_*.h5")  
print("   - Multiple years combined")
print("   - Better state-level sample sizes")
print("   - Ideal for geographic analysis")
print("")
print("View all datasets: https://huggingface.co/policyengine/policyengine-us-data/tree/main")

=== Available PolicyEngine Datasets ===
Enhanced CPS (Recommended): enhanced_cps_2024.h5
   - Enhanced with IRS Public Use File data
   - Best for national economy-wide analysis
   - ~42k household records

Pooled Datasets: pooled_*.h5
   - Multiple years combined
   - Better state-level sample sizes
   - Ideal for geographic analysis

View all datasets: https://huggingface.co/policyengine/policyengine-us-data/tree/main


## 2. Understanding Dataset Selection


PolicyEngine offers multiple datasets optimized for different types of analysis. For economy-wide simulations, we recommend using the Enhanced CPS dataset.

In [39]:
# Create baseline microsimulation using Enhanced CPS
print("Creating baseline microsimulation...")
baseline = Microsimulation(dataset=ENHANCED_CPS)

print(f"=== Microsimulation Overview ===")
print(f"Using dataset: Enhanced CPS 2024")
print(f"Analysis period: 2025")

# The enhanced CPS contains ~42k household records
# Each represents multiple real households via weights
sample_weights = baseline.calculate("household_weight", period=2025)
print(f"Number of household records: {len(sample_weights):,}")

print("\n=== Single Variable Calculations ===")
print("Calculating individual variables...")

# Calculate key variables one by one using .calculate()
# Child Tax Credit
ctc_values = baseline.calculate("ctc_value", period=2025)
print(f"CTC variable type: {type(ctc_values)}")
print(f"CTC array shape: {ctc_values.shape}")
print(f"First 5 CTC values: {ctc_values[:5]}")

# SNAP benefits
snap_values = baseline.calculate("snap", period=2025)
print(f"First 5 SNAP values: {snap_values[:5]}")

# Household net income
net_income = baseline.calculate("household_net_income", period=2025)
print(f"First 5 net income values: {net_income[:5]}")

Creating baseline microsimulation...
=== Microsimulation Overview ===
Using dataset: Enhanced CPS 2024
Analysis period: 2025
Number of household records: 41,310

=== Single Variable Calculations ===
Calculating individual variables...
CTC variable type: <class 'microdf.generic.MicroSeries'>
CTC array shape: (56768,)
First 5 CTC values:    value        weight
0    0.0  14044.562500
1    0.0   1151.017212
2    0.0   1151.017212
3    0.0  11405.556641
4    0.0  11405.556641
First 5 SNAP values:          value        weight
0     0.000000  14044.562500
1     0.000000   1151.017212
2   281.995483  11405.556641
3  3524.943604  11405.556641
4   281.995483   3046.133301
First 5 net income values:            value        weight
0  103463.171875  14044.562500
1   87697.304688   1151.017212
2  111439.476562  11405.556641
3   92810.648438   3046.133301
4   52004.863281  11906.229492


## 3

In [40]:
print("=== Understanding Automatic Weighting ===")

# The .calculate() method automatically applies weights when you sum
total_ctc = baseline.calculate("ctc_value", period=2025).sum()
total_snap = baseline.calculate("snap", period=2025).sum()
total_net_income = baseline.calculate("household_net_income", period=2025).sum()

print(f"BASELINE TOTALS (Auto-weighted):")
print(f"   Total CTC: ${total_ctc/1e9:.1f} billion")
print(f"   Total SNAP: ${total_snap/1e9:.1f} billion")
print(f"   Total Net Income: ${total_net_income/1e9:.1f} billion")

# Show how many households benefit from each program
ctc_recipients = (baseline.calculate("ctc_value", period=2025) > 0).sum()
snap_recipients = (baseline.calculate("snap", period=2025) > 0).sum()

print(f"\nPROGRAM PARTICIPATION:")
print(f"   Households receiving CTC: {ctc_recipients:,.0f}")
print(f"   Households receiving SNAP: {snap_recipients:,.0f}")

# Calculate average benefits among recipients
avg_ctc_recipients = total_ctc / ctc_recipients
avg_snap_recipients = total_snap / snap_recipients

print(f"\nAVERAGE BENEFITS (among recipients):")
print(f"   Average CTC: ${avg_ctc_recipients:,.0f}")
print(f"   Average SNAP: ${avg_snap_recipients:,.0f}")

=== Understanding Automatic Weighting ===
BASELINE TOTALS (Auto-weighted):
   Total CTC: $115.7 billion
   Total SNAP: $83.4 billion
   Total Net Income: $15813.4 billion

PROGRAM PARTICIPATION:
   Households receiving CTC: 44,203,382
   Households receiving SNAP: 23,539,852

AVERAGE BENEFITS (among recipients):
   Average CTC: $2,617
   Average SNAP: $3,545


## 4

In [41]:
print("=== Creating CTC Expansion Reform ===")

# Create CTC expansion reform
ctc_expansion = Reform.from_dict({
    "gov.irs.credits.ctc.amount.base[0].amount": {
        "2025-01-01.2100-12-31": 3600  # Increase from $2,000 to $3,600
    }
}, country_id="us")

print("Policy Reform Details:")
print("   Current CTC: $2,000 per child")
print("   Reformed CTC: $3,600 per child")
print("   Increase: $1,600 per child")

# Create reformed microsimulation
print("\nCreating reformed microsimulation...")
reformed = Microsimulation(reform=ctc_expansion, dataset=ENHANCED_CPS)

print("=== Baseline vs Reform Comparison ===")

# Calculate baseline and reformed totals using .calculate()
baseline_total_ctc = baseline.calculate("ctc_value", period=2025).sum()
reformed_total_ctc = reformed.calculate("ctc_value", period=2025).sum()
ctc_increase = reformed_total_ctc - baseline_total_ctc

print(f"AGGREGATE IMPACT:")
print(f"   Baseline total CTC: ${baseline_total_ctc/1e9:.1f} billion")
print(f"   Reformed total CTC: ${reformed_total_ctc/1e9:.1f} billion") 
print(f"   Annual increase: ${ctc_increase/1e9:.1f} billion")

# Calculate net income impact
baseline_net_income = baseline.calculate("household_net_income", period=2025).sum()
reformed_net_income = reformed.calculate("household_net_income", period=2025).sum()
net_income_increase = reformed_net_income - baseline_net_income

print(f"\nHOUSEHOLD IMPACT:")
print(f"   Total net income increase: ${net_income_increase/1e9:.1f} billion")
print(f"   This represents the total additional purchasing power")

# Show impact on program participation
baseline_ctc_recipients = (baseline.calculate("ctc_value", period=2025) > 0).sum()
reformed_ctc_recipients = (reformed.calculate("ctc_value", period=2025) > 0).sum()
recipient_change = reformed_ctc_recipients - baseline_ctc_recipients

print(f"\nPARTICIPATION CHANGES:")
print(f"   Baseline CTC recipients: {baseline_ctc_recipients:,.0f}")
print(f"   Reformed CTC recipients: {reformed_ctc_recipients:,.0f}")
print(f"   Change in recipients: {recipient_change:+,.0f}")

=== Creating CTC Expansion Reform ===
Policy Reform Details:
   Current CTC: $2,000 per child
   Reformed CTC: $3,600 per child
   Increase: $1,600 per child

Creating reformed microsimulation...
=== Baseline vs Reform Comparison ===
AGGREGATE IMPACT:
   Baseline total CTC: $115.7 billion
   Reformed total CTC: $185.5 billion
   Annual increase: $69.8 billion

HOUSEHOLD IMPACT:
   Total net income increase: $62.0 billion
   This represents the total additional purchasing power

PARTICIPATION CHANGES:
   Baseline CTC recipients: 44,203,382
   Reformed CTC recipients: 44,387,859
   Change in recipients: +184,477


## 5.


In [42]:
print("=== Introduction to calculate_dataframe() Method ===")

# Key variables for comprehensive analysis
CORE_VARIABLES = [
    "household_id",
    "household_weight",        # How many households this record represents
    "household_net_income",    # After-tax income
    "employment_income",       # Pre-tax earnings
    "ctc_value",              # Child Tax Credit
    "snap",                   # SNAP benefits  
    "state_code",             # State location
    "household_size",         # Number of people
]

print("Variables we'll analyze:")
for var in CORE_VARIABLES:
    print(f"   - {var}")

# Calculate baseline data using calculate_dataframe
print("\nCalculating baseline variables using calculate_dataframe...")
baseline_df = baseline.calculate_dataframe(CORE_VARIABLES, map_to="household", period=2025)

print(f"Baseline DataFrame shape: {baseline_df.shape}")
print(f"DataFrame columns: {list(baseline_df.columns)}")
print("\nFirst few records:")
print(baseline_df.head())

print("\n=== Understanding Weights: calculate() vs calculate_dataframe() ===")

# METHOD 1: Using calculate() - PolicyEngine automatically applies weights
total_ctc_auto = baseline.calculate("ctc_value", period=2025).sum()
print(f"Total CTC (auto-weighted by calculate()): ${total_ctc_auto/1e9:.1f} billion")

# METHOD 2: Using pandas DataFrame - weights are NOT automatically applied
total_ctc_unweighted = baseline_df['ctc_value'].sum()
print(f"Total CTC (unweighted DataFrame): ${total_ctc_unweighted/1e9:.1f} billion")

# METHOD 3: Manual weighting with DataFrame
total_ctc_manual = (baseline_df['ctc_value'] * baseline_df['household_weight']).sum()
print(f"Total CTC (manually weighted): ${total_ctc_manual/1e9:.1f} billion")

# Verification: Methods 1 and 3 should match
print(f"\nVERIFICATION:")
print(f"   calculate() vs manual weighting difference: ${(total_ctc_auto - total_ctc_manual)/1e6:.1f} million")
print(f"   This should be close to zero!")

# Show the importance of weighting
weight_multiplier = total_ctc_manual / total_ctc_unweighted
print(f"\nCRITICAL INSIGHT:")
print(f"   Weighted total is {weight_multiplier:.1f}x larger than unweighted")
print(f"   This shows why proper weighting is essential for accurate estimates")

# Number of households receiving CTC
ctc_recipients_unweighted = (baseline_df['ctc_value'] > 0).sum()
ctc_recipients_weighted = baseline_df[baseline_df['ctc_value'] > 0]['household_weight'].sum()

print(f"\nCTC RECIPIENTS:")
print(f"   Unweighted count: {ctc_recipients_unweighted:,} records")
print(f"   Weighted count: {ctc_recipients_weighted:,.0f} actual households")
print(f"   Multiplier: {ctc_recipients_weighted/ctc_recipients_unweighted:.1f}x")

=== Introduction to calculate_dataframe() Method ===
Variables we'll analyze:
   - household_id
   - household_weight
   - household_net_income
   - employment_income
   - ctc_value
   - snap
   - state_code
   - household_size

Calculating baseline variables using calculate_dataframe...
Baseline DataFrame shape: (41310, 8)
DataFrame columns: ['household_id', 'household_weight', 'household_net_income', 'employment_income', 'ctc_value', 'snap', 'state_code', 'household_size']

First few records:
   household_id  household_weight  household_net_income  employment_income  \
0            12      14044.562500         103463.171875        4219.356445   
1            21       1151.017212          87697.304688       96693.578125   
2            22      11405.556641         111439.476562           0.000000   
3            30       3046.133301          92810.648438       87903.257812   
4            36      11906.229492          52004.863281       34985.497070   

   ctc_value         snap state

## 6

In [43]:
print("=== 10-Year Budgetary Impact Analysis ===")

# Calculate impacts for multiple years (2025-2034)
years = list(range(2025, 2035))
annual_impacts = []

print("Calculating annual impacts across 10 years...")
for year in years:
    # Calculate baseline and reform totals for each year
    baseline_ctc = baseline.calculate("ctc_value", period=year).sum()
    reformed_ctc = reformed.calculate("ctc_value", period=year).sum()
    annual_increase = reformed_ctc - baseline_ctc
    
    annual_impacts.append({
        'year': year,
        'baseline_ctc': baseline_ctc / 1e9,
        'reformed_ctc': reformed_ctc / 1e9,
        'annual_increase': annual_increase / 1e9
    })
    
    print(f"   {year}: ${annual_increase/1e9:.1f}B increase")

# Create DataFrame for analysis
impact_df = pd.DataFrame(annual_impacts)

# Calculate 10-year total
ten_year_total = impact_df['annual_increase'].sum()
print(f"\n10-YEAR FISCAL IMPACT:")
print(f"   Total 10-year CTC increase: ${ten_year_total:.1f} billion")
print(f"   Average annual cost: ${ten_year_total/10:.1f} billion")

# Calculate compound annual growth rate
first_year = impact_df['annual_increase'].iloc[0]
last_year = impact_df['annual_increase'].iloc[-1]
if first_year > 0:
    cagr = ((last_year / first_year) ** (1/9) - 1) * 100
    print(f"   Compound annual growth rate: {cagr:.1f}%")

=== 10-Year Budgetary Impact Analysis ===
Calculating annual impacts across 10 years...
   2025: $69.8B increase
   2026: $102.1B increase
   2027: $102.2B increase
   2028: $101.8B increase
   2029: $101.1B increase
   2030: $100.1B increase
   2031: $99.1B increase
   2032: $97.9B increase
   2033: $96.6B increase
   2034: $95.1B increase

10-YEAR FISCAL IMPACT:
   Total 10-year CTC increase: $965.9 billion
   Average annual cost: $96.6 billion
   Compound annual growth rate: 3.5%


## 7


In [44]:
print("=== Using DataFrames for Reform Analysis ===")

# Calculate the same variables under reform
reformed_df = reformed.calculate_dataframe(CORE_VARIABLES, map_to="household", period=2025)

print("Creating comparison analysis...")

# Method 1: Using calculate() for verification
baseline_total_manual = (baseline_df['ctc_value'] * baseline_df['household_weight']).sum()
reformed_total_manual = (reformed_df['ctc_value'] * reformed_df['household_weight']).sum()

print(f"DATAFRAME COMPARISON (with manual weighting):")
print(f"   Baseline (manual): ${baseline_total_manual/1e9:.1f} billion")
print(f"   Reformed (manual): ${reformed_total_manual/1e9:.1f} billion")
print(f"   Difference: ${(reformed_total_manual - baseline_total_manual)/1e9:.1f} billion")

# Verify against single variable method
print(f"\nVERIFICATION against calculate() method:")
print(f"   Single variable difference: ${ctc_increase/1e9:.1f} billion")
print(f"   DataFrame difference: ${(reformed_total_manual - baseline_total_manual)/1e9:.1f} billion")
print(f"   Match? {abs(ctc_increase - (reformed_total_manual - baseline_total_manual)) < 1e6}")

print("\n=== Geographic Analysis by State ===")

# Create comparison DataFrame
print("Preparing state-level analysis...")
comparison_df = baseline_df.copy()
comparison_df['ctc_baseline'] = baseline_df['ctc_value']
comparison_df['ctc_reformed'] = reformed_df['ctc_value']
comparison_df['ctc_increase'] = comparison_df['ctc_reformed'] - comparison_df['ctc_baseline']

# Group by state and calculate weighted totals
print("Calculating state-level impacts...")
state_analysis = comparison_df.groupby('state_code').apply(
    lambda x: pd.Series({
        'households': x['household_weight'].sum(),
        'total_ctc_increase': (x['ctc_increase'] * x['household_weight']).sum(),
        'avg_ctc_increase': (x['ctc_increase'] * x['household_weight']).sum() / x['household_weight'].sum(),
        'households_benefiting': x[x['ctc_increase'] > 0]['household_weight'].sum()
    })
).reset_index()

# Calculate percentage of households benefiting
state_analysis['pct_households_benefiting'] = (
    state_analysis['households_benefiting'] / state_analysis['households'] * 100
)

# Sort by total CTC increase
state_analysis = state_analysis.sort_values('total_ctc_increase', ascending=False)

print(f"\nTOP 10 STATES BY TOTAL CTC INCREASE:")
top_states = state_analysis.head(10)
for _, row in top_states.iterrows():
    print(f"   {row['state_code']}: ${row['total_ctc_increase']/1e9:.1f}B total, "
          f"${row['avg_ctc_increase']:,.0f} avg, "
          f"{row['pct_households_benefiting']:.1f}% benefit")

print(f"\nSTATES WITH HIGHEST BENEFIT RATES:")
top_pct_states = state_analysis.nlargest(5, 'pct_households_benefiting')
for _, row in top_pct_states.iterrows():
    print(f"   {row['state_code']}: {row['pct_households_benefiting']:.1f}% of households benefit")

=== Using DataFrames for Reform Analysis ===
Creating comparison analysis...
DATAFRAME COMPARISON (with manual weighting):
   Baseline (manual): $115.7 billion
   Reformed (manual): $13322317.9 billion
   Difference: $13322202.2 billion

VERIFICATION against calculate() method:
   Single variable difference: $69.8 billion
   DataFrame difference: $13322202.2 billion
   Match? False

=== Geographic Analysis by State ===
Preparing state-level analysis...
Calculating state-level impacts...

TOP 10 STATES BY TOTAL CTC INCREASE:
   CA: $7.1B total, $386 avg, 21.5% benefit
   TX: $6.5B total, $527 avg, 24.7% benefit
   FL: $4.1B total, $369 avg, 17.5% benefit
   NY: $3.6B total, $387 avg, 19.8% benefit
   PA: $2.8B total, $489 avg, 19.1% benefit
   OH: $2.7B total, $508 avg, 20.3% benefit
   NJ: $2.4B total, $615 avg, 24.9% benefit
   MI: $2.4B total, $505 avg, 23.1% benefit
   GA: $2.2B total, $456 avg, 23.7% benefit
   IL: $2.1B total, $365 avg, 17.1% benefit

STATES WITH HIGHEST BENEFIT R

## 8


In [45]:
print("=== Program Enrollment Analysis ===")

# Analyze SNAP enrollment (map_to="person" for individual-level analysis)
PERSON_VARIABLES = [
    "person_id",
    "person_weight", 
    "age",
    "snap"
]

print("Calculating person-level data...")
baseline_persons = baseline.calculate_dataframe(PERSON_VARIABLES, map_to="person", period=2025)
reformed_persons = reformed.calculate_dataframe(PERSON_VARIABLES, map_to="person", period=2025)

print(f"Person-level dataset shape: {baseline_persons.shape}")

# SNAP enrollment: people with SNAP > 0
baseline_snap_enrolled = baseline_persons[baseline_persons['snap'] > 0]
reformed_snap_enrolled = reformed_persons[reformed_persons['snap'] > 0]

# Apply person weights for accurate counts
baseline_snap_count = baseline_snap_enrolled['person_weight'].sum()
reformed_snap_count = reformed_snap_enrolled['person_weight'].sum()

print(f"\nSNAP ENROLLMENT ANALYSIS:")
print(f"   Baseline SNAP enrollment: {baseline_snap_count:,.0f} people")
print(f"   Reformed SNAP enrollment: {reformed_snap_count:,.0f} people")
print(f"   Change in SNAP enrollment: {reformed_snap_count - baseline_snap_count:+,.0f} people")

# Children analysis
baseline_children = baseline_persons[baseline_persons['age'] < 18]
reformed_children = reformed_persons[reformed_persons['age'] < 18]

total_children = baseline_children['person_weight'].sum()
print(f"\nCHILDREN ANALYSIS:")
print(f"   Total children in dataset: {total_children:,.0f}")
print(f"   Note: CTC eligibility depends on household structure and income")

# Percentage change in SNAP enrollment
if baseline_snap_count > 0:
    snap_change_pct = ((reformed_snap_count - baseline_snap_count) / baseline_snap_count) * 100
    print(f"   SNAP enrollment change: {snap_change_pct:+.2f}%")

# Dataset selection information
print("\n=== Dataset Selection for Different Analysis Types ===")

print("Dataset Recommendations:")
print("   • National analysis: enhanced_cps_2024.h5 (current choice)")
print("   • State-level analysis: Consider pooled datasets for larger samples")
print("   • Historical analysis: Use year-specific datasets")

# Demonstrate state sample sizes with enhanced CPS
state_samples = baseline_df.groupby('state_code').agg({
    'household_weight': ['count', 'sum']
}).round()

state_samples.columns = ['Records', 'Weighted_Households']
state_samples = state_samples.sort_values('Weighted_Households', ascending=False)

print(f"\nSTATE SAMPLE SIZE INFORMATION:")
print(f"   Average records per state: {state_samples['Records'].mean():.0f}")
print(f"   States with <100 records: {(state_samples['Records'] < 100).sum()}")
print(f"   For detailed state analysis, consider pooled datasets")

=== Program Enrollment Analysis ===
Calculating person-level data...
Person-level dataset shape: (101726, 4)

SNAP ENROLLMENT ANALYSIS:
   Baseline SNAP enrollment: 2,994,233,014,168 people
   Reformed SNAP enrollment: 2,994,233,014,168 people
   Change in SNAP enrollment: +0 people

CHILDREN ANALYSIS:
   Total children in dataset: 4,587,869,965,050
   Note: CTC eligibility depends on household structure and income
   SNAP enrollment change: +0.00%

=== Dataset Selection for Different Analysis Types ===
Dataset Recommendations:
   • National analysis: enhanced_cps_2024.h5 (current choice)
   • State-level analysis: Consider pooled datasets for larger samples
   • Historical analysis: Use year-specific datasets

STATE SAMPLE SIZE INFORMATION:
   Average records per state: 810
   States with <100 records: 0
   For detailed state analysis, consider pooled datasets


## 9.


In [46]:
# Create multiple visualizations

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('10-Year Impact Trend', 'State-by-State Impact', 
                   'CTC Distribution Comparison', 'Program Enrollment'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Plot 1: 10-year trend
fig.add_trace(
    go.Scatter(x=impact_df['year'], y=impact_df['annual_increase'],
              mode='lines+markers', name='Annual CTC Increase',
              line=dict(color='blue', width=3)),
    row=1, col=1
)

# Plot 2: Top 10 states
top_10_states = state_analysis.head(10)
fig.add_trace(
    go.Bar(x=top_10_states['state_code'], y=top_10_states['total_ctc_increase']/1e9,
           name='State CTC Increase', marker_color='green'),
    row=1, col=2
)

# Plot 3: CTC distribution comparison
ctc_bins = np.arange(0, 4000, 200)
baseline_hist = np.histogram(comparison_df['ctc_baseline'], bins=ctc_bins, 
                           weights=comparison_df['household_weight'])[0]
reformed_hist = np.histogram(comparison_df['ctc_reformed'], bins=ctc_bins,
                           weights=comparison_df['household_weight'])[0]

fig.add_trace(
    go.Bar(x=ctc_bins[:-1], y=baseline_hist/1e6, name='Baseline CTC',
           marker_color='red', opacity=0.6),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=ctc_bins[:-1], y=reformed_hist/1e6, name='Reformed CTC',
           marker_color='blue', opacity=0.6),
    row=2, col=1
)

# Plot 4: Program enrollment comparison
enrollment_data = {
    'Program': ['SNAP (Baseline)', 'SNAP (Reformed)'],
    'Enrollment': [baseline_snap_count/1e6, reformed_snap_count/1e6]
}
fig.add_trace(
    go.Bar(x=enrollment_data['Program'], y=enrollment_data['Enrollment'],
           name='SNAP Enrollment', marker_color='orange'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=800, showlegend=True, 
                 title_text="CTC Reform: Comprehensive Policy Impact Analysis")

# Update axis labels
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Annual Increase ($B)", row=1, col=1)
fig.update_xaxes(title_text="State", row=1, col=2)
fig.update_yaxes(title_text="Total Increase ($B)", row=1, col=2)
fig.update_xaxes(title_text="CTC Amount ($)", row=2, col=1)
fig.update_yaxes(title_text="Households (Millions)", row=2, col=1)
fig.update_xaxes(title_text="Program", row=2, col=2)
fig.update_yaxes(title_text="Enrollment (Millions)", row=2, col=2)

fig.show()

Summary
This notebook demonstrated a progressive approach to economy-wide policy analysis using PolicyEngine:

Foundation: Started with single variable calculations using .calculate()
Core Concept: Understood automatic weighting in .calculate()
Advanced Method: Introduced calculate_dataframe() for multiple variables
Critical Insight: Learned the difference between automatic and manual weighting
Comprehensive Analysis: Applied concepts to geographic, temporal, and program analysis

Key Learning Progression: Single variables → automatic weighting → DataFrames → manual weighting → complex analysis
This training material provides a solid foundation for conducting rigorous economy-wide policy analysis using PolicyEngine's microsimulation capabilities.